In [1]:
import pandas as pd
import numpy as np

np.random.seed(0)

In [2]:
data = pd.read_csv('LBW_Dataset.csv')

In [3]:
data.head(5)

In [4]:
data.info()

In [5]:
data.isnull().sum()

In [6]:
for col in data.columns:
    print("{}:{}".format(col,data[col].nunique()))

In [7]:
data.describe()


In [8]:
data['Community'].value_counts()
#NO null values in community column

In [9]:
data['Residence'].value_counts()

1.0    81
2.0    13
Name: Residence, dtype: int64

In [10]:
# FIll Age ,Weight ,HB and BP  Null values with mean
# Delivery phase with 1 since the coulmn takes binary values
# Education with 5
#Residence with 1

In [11]:
data.columns

Index(['Community', 'Age', 'Weight', 'Delivery phase', 'HB', 'IFA', 'BP',
       'Education', 'Residence', 'Result'],
      dtype='object')

In [12]:
x=data['HB'].mean()
print(x)

9.076623376623376


In [13]:
values = { 'Age':data['Age'].mean(), 'Weight':data['Weight'].mean(), 'Delivery phase':1, 'HB':9.0, 'BP':data['BP'].mean(),
       'Education':5, 'Residence':1}

In [14]:
data = data.fillna(value=values)

In [15]:
data.head(3)

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Education,Residence,Result
0,1,21.0,42.000000,1.0,9.2,1,1.375,5.0,1.0,0
1,1,21.0,45.129412,1.0,8.8,1,1.500,5.0,1.0,0
2,1,21.0,45.129412,1.0,9.2,1,2.125,5.0,1.0,0


In [16]:
data.isnull().sum()

Community         0
Age               0
Weight            0
Delivery phase    0
HB                0
IFA               0
BP                0
Education         0
Residence         0
Result            0
dtype: int64

In [17]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
#activation function

In [18]:
data['Result'].value_counts()


1    72
0    24
Name: Result, dtype: int64

In [19]:
data=data.drop(columns='Education')


In [20]:
#75% of the result column is 1 so the model might over fit 

In [21]:
#

In [22]:
class ANN_layers:
    def __init__(self,num_inputs ,num_cells):
        self.weights = 0.1 * np.random.randn(num_inputs,num_cells) #in range[-1,1]
        self.biases = np.zeros((1,num_cells))
    def fpropogation(self , inputs):
        self.inputs= inputs
        self.output= np.dot(inputs ,self.weights)
    def Bpropogation(self,dval):
        self.dweights= np.dot(self.inputs.T,dval)
        self.dbiases = np.sum(dval,axis=0,keepdims=True)
        self.dinputs =np.dot(dval,self.weights.T)

def confidenceTolist(x):
    rows = x.shape[0]
    cols = x.shape[1]
    predict=[None]*rows
    for i in range(0,rows):
        if(x[i,0]>x[i,1]):
            predict[i]=0
        else:
            predict[i]=1
    return predict
        
        
    
        
class activation:
    def fRelu(self,x):
        self.inputs=x
        self.output = np.maximum(0,x)
        #sigmoid
        #self.inputs =x
        #self.output = 1/(1+np.exp(-x))
        #self.output = np.tanh(x) #tanh
    def sigmoid(self,x):
        self.inputs =x
        self.output = 1/(1+np.exp(-x))
    
    def tanh(self,x):
        self.inputs =x
        self.output = np.tanh(x)
        
    def back(self,dval):
        self.dinputs = dval.copy()
        self.dinputs[self.inputs <=0] =0 
        

class act_softmax :       
    def fsoftmax(self,inputs):
        self.inputs = inputs
        exp1 = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        prob = exp1 /np.sum(exp1 ,axis=1, keepdims=True)
        self.output = prob
        
    def bsoftmax(self,dval):
        self.dinputs = np.empty_like(dval)
        
        for i ,(single_o,single_dval) in enumerate(zip(self.output,dval)):
            single_o = single_o.reshape(-1,1)
            jacobian = np.diagflat(single_o)-np.dot(single_o,single_o.T)
            self.dinputs[i] = np.dot(jacobian,single_dval)
        
class loss:
    def cal(self,output,y):
        sampleLosses = self.check(output,y)
        data_loss= np.mean(sampleLosses)
        return data_loss
    
class CCE(loss):
    def check(self,y_pred,y_true):
        x = len(y_pred)
        y_pred_clipped = np.clip(y_pred ,1e-7,1 - 1e-7)
        confidence = y_pred_clipped[range(x),y_true]
        negLikely = -np.log(confidence)
        return negLikely
    
    def backward(self,dval,y_true):
        
        x=len(dval)
        label=len(dval[0])
        if len(y_true.shape)==1:
            y_true=np.eye(label)[y_true] #
        self.dinputs =-y_true/dval
        self.dinputs =self.dinputs/x


In [23]:
class optimizer:
    
    def __init__(self , l_rate=1.,decay=0.,p=0.):
        self.l_rate = l_rate
        self.curr_l_rate=l_rate
        self.decay = decay
        self.iter =0
        self.p =p
        
    def initial_update_param(self):
        if self.decay:
            self.curr_l_rate = self.l_rate *(1./(1+self.decay*self.iter))
            
    def update_params(self,layer):
        if self.p:
            if not hasattr(layer,'wieght_momentums'):
                layer.weight_p = np.zeros_like(layer.weights)
                layer.bias_p = np.zeros_like(layer.biases)
            
            w_updates= self.p*layer.weight_p - self_curr_l_rate*layer.dweights
            layer.weight_p = w_updates
            
            b_updates= self.p*layer.bias_p - self_curr_l_rate*layer.dbiases
            layer.bias_p = b_updates
        
        else:
            w_updates =-self.curr_l_rate * layer.dweights
            b_updates =-self.curr_l_rate * layer.dbiases
            
        layer.weights+=w_updates
        layer.biases+=b_updates
        
    def post_update_params(self):
        self.iter+=1

In [24]:
class Activation_softmax_LCCE():
    
    def __init__(self):
        self.activation=act_softmax()
        self.loss=CCE()
    
    def forward(self,inputs,y_true):
        self.activation.fsoftmax(inputs)
        self.output=self.activation.output
        
        return self.loss.cal(self.output,y_true)
        
    def backpass(self,dval,y_true):
        x=len(dval)
        if len(y_true.shape)==2:
            y_true = np.argmax(y_true,axis=1)
        
        self.dinputs =dval.copy()
        self.dinputs[range(x),y_true]-=1
        self.dinputs= self.dinputs/x

In [25]:
input2=data.loc[:,['Age', 'Weight', 'Delivery phase', 'HB', 'IFA', 'BP', 'Residence']] 

In [26]:
result=data['Result']

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
input1,x_test, result1, y_test = train_test_split(input2, result, test_size=0.35, random_state=42)

In [29]:
Hlayer1=ANN_layers(7,16)

In [30]:
Hlayer2=ANN_layers(16,16)

In [31]:
Output_layer=ANN_layers(16,2)

In [32]:
act1=activation()

In [33]:
act2=activation()


In [34]:
loss_activation=Activation_softmax_LCCE()

In [35]:
optimize = optimizer(l_rate=0.025,decay=5e-7)

In [36]:
for epoch in range(10001):
    Hlayer1.fpropogation(input1)
    act1.fRelu(Hlayer1.output)
    Hlayer2.fpropogation(act1.output)
    act2.fRelu(Hlayer2.output)
    Output_layer.fpropogation(act2.output)
    loss = loss_activation.forward(Output_layer.output,result1)
    prediction = np.argmax(loss_activation.output,axis=1)
    if len(result1.shape)==2:
        result1=np.argmax(result1,axis=1)
    accuracy= np.mean(prediction==result1)
    if not epoch %100:
        print(f'epoch:{epoch} ,'+
              f'accuracy:{accuracy:.3f} ,'+
              f'loss:{loss} ,'+
              f'lr:{optimize.curr_l_rate},')
    loss_activation.backpass(loss_activation.output,result1)
    Output_layer.Bpropogation(loss_activation.dinputs)
    act2.back(Output_layer.dinputs)
    Hlayer2.Bpropogation(act2.dinputs)
    act1.back(Hlayer2.dinputs)
    Hlayer1.Bpropogation(act1.dinputs)
              
    optimize.initial_update_param()
    optimize.update_params(Hlayer1)
    optimize.update_params(Hlayer2)
    optimize.update_params(Output_layer)
    optimize.post_update_params()

epoch:0 ,accuracy:0.339 ,loss:0.7339040423240516 ,lr:0.025,
epoch:100 ,accuracy:0.790 ,loss:0.4895713225451092 ,lr:0.02499876256125322,
epoch:200 ,accuracy:0.790 ,loss:0.4811253595825717 ,lr:0.02499751274748163,
epoch:300 ,accuracy:0.806 ,loss:0.4752756782995178 ,lr:0.024996263058672728,
epoch:400 ,accuracy:0.806 ,loss:0.4693595196997094 ,lr:0.02499501349480779,
epoch:500 ,accuracy:0.806 ,loss:0.4676429697003342 ,lr:0.024993764055868062,
epoch:600 ,accuracy:0.806 ,loss:0.4590094708405892 ,lr:0.024992514741834823,
epoch:700 ,accuracy:0.806 ,loss:0.45483294041106703 ,lr:0.024991265552689337,
epoch:800 ,accuracy:0.806 ,loss:0.45197723769086795 ,lr:0.024990016488412876,
epoch:900 ,accuracy:0.806 ,loss:0.4474648582545687 ,lr:0.024988767548986733,
epoch:1000 ,accuracy:0.823 ,loss:0.4467685093118895 ,lr:0.024987518734392172,
epoch:1100 ,accuracy:0.806 ,loss:0.45369690506086163 ,lr:0.024986270044610487,
epoch:1200 ,accuracy:0.806 ,loss:0.4442178014321335 ,lr:0.024985021479622966,
epoch:1300 ,a

In [37]:
Hlayer1.fpropogation(x_test)
act1.fRelu(Hlayer1.output)
Hlayer2.fpropogation(act1.output)
act2.fRelu(Hlayer2.output)
Output_layer.fpropogation(act2.output)
loss = loss_activation.forward(Output_layer.output,y_test)
prediction = np.argmax(loss_activation.output,axis=1)
if len(y_test.shape)==2:
    result1=np.argmax(y_test,axis=1)
accuracy= np.mean(prediction==y_test)
print(f'epoch:{epoch} ,'+f'accuracy:{accuracy:.3f} ,'+
      f'lr:{optimize.curr_l_rate},')

epoch:10000 ,accuracy:0.765 ,lr:0.02487562189054727,


In [38]:
#since dataset is biased towards 1 we will try SMOTE approach

In [39]:
data2=data[(data['Result']==0)]
data2.head(3)

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Residence,Result
0,1,21.0,42.000000,1.0,9.2,1,1.375,1.0,0
1,1,21.0,45.129412,1.0,8.8,1,1.500,1.0,0
2,1,21.0,45.129412,1.0,9.2,1,2.125,1.0,0


In [40]:
new_data=pd.concat([data,data2])
#new_data=pd.concat([new_data,data2])
new_data.head(3)

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Residence,Result
0,1,21.0,42.000000,1.0,9.2,1,1.375,1.0,0
1,1,21.0,45.129412,1.0,8.8,1,1.500,1.0,0
2,1,21.0,45.129412,1.0,9.2,1,2.125,1.0,0


In [41]:
new_data=new_data.sample(frac=1)
new_data.head(3)

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Residence,Result
61,1,25.0,41.0,1.0,9.1,0,1.375000,1.0,1
60,1,21.0,40.0,1.0,7.9,0,1.666667,1.0,1
84,1,26.0,44.0,1.0,9.0,0,1.725189,1.0,1


In [42]:
new_data['Result'].value_counts()

1    72
0    48
Name: Result, dtype: int64

In [43]:
new_input=new_data.loc[:,['Age', 'Weight', 'Delivery phase', 'HB', 'IFA', 'BP', 'Residence']]
new_result=new_data['Result']
input11,x_test11, result11, y_test11 = train_test_split(new_input, new_result, test_size=0.35, random_state=42)
Hlayer11=ANN_layers(7,128)
#Hlayer12=ANN_layers(16,8)
Output_layer1=ANN_layers(128,2)
act11=activation()
#act12=activation()
loss_activation1=Activation_softmax_LCCE()
optimize1 = optimizer(l_rate=0.05,decay=5e-7)
for epoch in range(11001):
    Hlayer11.fpropogation(input11)
    act11.fRelu(Hlayer11.output)
    #Hlayer12.fpropogation(act11.output)
    #act12.fRelu(Hlayer12.output)
    Output_layer1.fpropogation(act11.output)
    loss1 = loss_activation1.forward(Output_layer1.output,result11)
    prediction1 = np.argmax(loss_activation1.output,axis=1)
    if len(result11.shape)==2:
        result11=np.argmax(result11,axis=1)
    accuracy1= np.mean(prediction1==result11)
    if not epoch %100:
        print(f'epoch:{epoch} ,'+
              f'accuracy:{accuracy1:.3f} ,'+
              f'loss:{loss1:.3f} ,'+
              f'lr:{optimize.curr_l_rate},')
        
    loss_activation1.backpass(loss_activation1.output,result11)
    Output_layer1.Bpropogation(loss_activation1.dinputs)
    act11.back(Output_layer1.dinputs)
    #Hlayer12.Bpropogation(act12.dinputs)
    #act11.back(Hlayer12.dinputs)
    Hlayer11.Bpropogation(act11.dinputs)
              
    optimize.initial_update_param()
    optimize.update_params(Hlayer11)
    #optimize.update_params(Hlayer12)
    optimize.update_params(Output_layer1)
    optimize.post_update_params()

epoch:0 ,accuracy:0.410 ,loss:2.023 ,lr:0.02487562189054727,
epoch:100 ,accuracy:0.679 ,loss:0.593 ,lr:0.024874384358987117,
epoch:200 ,accuracy:0.718 ,loss:0.593 ,lr:0.02487314695055218,
epoch:300 ,accuracy:0.782 ,loss:0.571 ,lr:0.024871909665224096,
epoch:400 ,accuracy:0.744 ,loss:0.552 ,lr:0.02487067250298448,
epoch:500 ,accuracy:0.756 ,loss:0.561 ,lr:0.02486943546381497,
epoch:600 ,accuracy:0.756 ,loss:0.538 ,lr:0.024868198547697205,
epoch:700 ,accuracy:0.744 ,loss:0.541 ,lr:0.024866961754612824,
epoch:800 ,accuracy:0.731 ,loss:0.561 ,lr:0.024865725084543464,
epoch:900 ,accuracy:0.744 ,loss:0.521 ,lr:0.024864488537470787,
epoch:1000 ,accuracy:0.756 ,loss:0.541 ,lr:0.02486325211337643,
epoch:1100 ,accuracy:0.756 ,loss:0.533 ,lr:0.02486201581224206,
epoch:1200 ,accuracy:0.795 ,loss:0.507 ,lr:0.024860779634049326,
epoch:1300 ,accuracy:0.769 ,loss:0.517 ,lr:0.024859543578779897,
epoch:1400 ,accuracy:0.756 ,loss:0.513 ,lr:0.02485830764641543,
epoch:1500 ,accuracy:0.808 ,loss:0.497 ,lr:0

In [44]:
x_test=x_test11
y_test=y_test11
Hlayer11.fpropogation(x_test)
act11.fRelu(Hlayer11.output)
#Hlayer12.fpropogation(act11.output)
#act12.fRelu(Hlayer12.output)
Output_layer1.fpropogation(act11.output)
loss1 = loss_activation1.forward(Output_layer1.output,y_test)
prediction1 = np.argmax(loss_activation1.output,axis=1)
if len(y_test.shape)==2:
    result11=np.argmax(y_test,axis=1)
accuracy1= np.mean(prediction1==y_test)
print(f'epoch:{epoch} ,'+f'accuracy:{accuracy1:.3f} ,'+
       f'loss:{loss1:.3f} ,'+
      f'lr:{optimize.curr_l_rate}')

epoch:11000 ,accuracy:0.881 ,loss:0.547 ,lr:0.024740215368522826
